In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
# from langchain.chat_models import init_chat_model
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import hub
from langgraph.prebuilt import create_react_agent

In [2]:
db = SQLDatabase.from_uri("sqlite:///data/northwind.db")
print("Database loaded successfully.", db.dialect)
print("Database tables:", db.get_usable_table_names())



Database loaded successfully. sqlite
Database tables: ['Categories', 'CustomerCustomerDemo', 'CustomerDemographics', 'Customers', 'EmployeeTerritories', 'Employees', 'Order Details', 'Orders', 'Products', 'Regions', 'Shippers', 'Suppliers', 'Territories']


In [3]:
## testing to see sql can be operated
print("Testing SQL query...")
result = db.run("SELECT * FROM Customers LIMIT 5")
print("Query result:", result)


Testing SQL query...
Query result: [('ALFKI', 'Alfreds Futterkiste', 'Maria Anders', 'Sales Representative', 'Obere Str. 57', 'Berlin', 'Western Europe', '12209', 'Germany', '030-0074321', '030-0076545'), ('ANATR', 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Owner', 'Avda. de la Constitución 2222', 'México D.F.', 'Central America', '05021', 'Mexico', '(5) 555-4729', '(5) 555-3745'), ('ANTON', 'Antonio Moreno Taquería', 'Antonio Moreno', 'Owner', 'Mataderos  2312', 'México D.F.', 'Central America', '05023', 'Mexico', '(5) 555-3932', None), ('AROUT', 'Around the Horn', 'Thomas Hardy', 'Sales Representative', '120 Hanover Sq.', 'London', 'British Isles', 'WA1 1DP', 'UK', '(171) 555-7788', '(171) 555-6750'), ('BERGS', 'Berglunds snabbköp', 'Christina Berglund', 'Order Administrator', 'Berguvsvägen  8', 'Luleå', 'Northern Europe', 'S-958 22', 'Sweden', '0921-12 34 65', '0921-12 34 67')]


In [10]:
from dotenv import load_dotenv
load_dotenv()

## Load the LLM
llm = ChatGoogleGenerativeAI(model= "gemini-pro-latest")
## initialize the SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
print("Toolkit initialized successfully.")
tools = toolkit.get_tools()
print("Available tools:", tools)


Toolkit initialized successfully.
Available tools: [QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10a7af620>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10a7af620>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10a7af620>), QuerySQLCheckerTool(des

In [11]:
## prompt templeate for nl2SQL
prompt_template = hub.pull('langchain-ai/sql-agent-system-prompt')
print(prompt_template)

/Users/shankuriakose/GitHub/NaturalLanguage2SQL_agent/venv/lib/python3.13/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['dialect', 'top_k'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-agent-system-prompt', 'lc_hub_commit_hash': '31156d5fe3945188ee172151b086712d22b8c70f8f1c0505f5457594424ed352'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\

In [12]:
prompt_template.messages[0].pretty_print()

================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [13]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)
print("System message:", system_message)

System message: System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the d

In [14]:
## create the agent
sql_agent = create_react_agent(
    llm,
    tools,
    prompt=system_message,

)

In [ ]:
query = "Which customer has spent the most in total purchases?"

# stream the events
for event in sql_agent.stream(
    {"messages": ('user', query) },
    stream_mode= 'values'
):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

identify the ten most expensive products(name, unit price). ?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..


NotFound: 404 models/gemini-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [33]:
llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash-latest")

In [34]:
llm.model

'models/gemini-1.5-flash-latest'

In [35]:
## create the agent
sql_agent = create_react_agent(
    llm,
    tools,
    prompt=system_message,

)

In [ ]:
query = "identify the expensive products(name, unit price). ?"

# stream the events
for event in sql_agent.stream(
    {"messages": ('user', query) },
    stream_mode= 'values'
):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

identify the ten most expensive products(name, unit price). ?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (e630a6f6-8d1a-46f3-9fb1-2911b7656d06)
 Call ID: e630a6f6-8d1a-46f3-9fb1-2911b7656d06
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (5625b8d4-2833-49a7-8e72-e7c43b5b002b)
 Call ID: 5625b8d4-2833-49a7-8e72-e7c43b5b002b
  Args:
    table_names: Products
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Products" (
	"Product